In [4]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**model training Notebook** - [here](https://www.kaggle.com/code/asheniranga/paddy-doctor-training)  

## predictions of each model

In [2]:
s1 = pd.read_csv('../input/paddydocoutputs/model_submission_v5.csv')
s2 = pd.read_csv('../input/paddy-doctor-training/model_submission_v5.csv')
s3 = pd.read_csv('../input/k/kasunpramodya/paddy-doctor-training/model_submission_v5.csv')

## Load models

In [5]:
m1 = tf.keras.models.load_model('../input/paddydocoutputs/model_xception.hdf5', custom_objects={'KerasLayer':hub.KerasLayer})
m3 = tf.keras.models.load_model('../input/k/kasunpramodya/paddy-doctor-training/model_effecientV2_b4.hdf5', custom_objects={'KerasLayer':hub.KerasLayer})

2022-07-07 03:59:27.145282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 03:59:27.250283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 03:59:27.250993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 03:59:27.252615: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
back_bone = tf.keras.applications.Xception(weights=None, input_shape=(224,224,3), include_top=False)
input_layer = tf.keras.layers.Input(shape=(224,224,3))
x = back_bone(input_layer)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output_layer = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.models.Model(input_layer,output_layer)
model.load_weights('../input/paddy-doctor-training/model_xception_weights.hdf5')

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

## Dataloader

In [7]:
test_loc = '../input/paddy-disease-classification/test_images'

test_data_224 = ImageDataGenerator(rescale=1.0/255).flow_from_directory(directory=test_loc,
                                                                    target_size=(224, 224),
                                                                    batch_size=32,
                                                                    classes=['.'],
                                                                    shuffle=False,
                                                                   )

test_data_300 = ImageDataGenerator(rescale=1.0/255).flow_from_directory(directory=test_loc,
                                                                    target_size=(300, 300),
                                                                    batch_size=16,
                                                                    classes=['.'],
                                                                    shuffle=False,
                                                                   )

Found 3469 images belonging to 1 classes.
Found 3469 images belonging to 1 classes.


## Inferance

In [8]:
m1_p = m1.predict(test_data_224, verbose=1)
m2_p = model.predict(test_data_224, verbose=1)
m3_p = m3.predict(test_data_300, verbose=1)

m1_predict_max = np.argmax(m1_p,axis=1)
m1_confidance = np.max(m1_p,axis=1)

m2_predict_max = np.argmax(m2_p,axis=1)
m2_confidance = np.max(m2_p,axis=1)

m3_predict_max = np.argmax(m3_p,axis=1)
m3_confidance = np.max(m3_p,axis=1)

2022-07-07 04:00:27.036768: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-07 04:00:30.308306: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


217/217 [==============================] - 29s 123ms/step


In [18]:
class_indices = {'bacterial_leaf_blight': 0,
                 'bacterial_leaf_streak': 1,
                 'bacterial_panicle_blight': 2,
                 'blast': 3,
                 'brown_spot': 4,
                 'dead_heart': 5,
                 'downy_mildew': 6,
                 'hispa': 7,
                 'normal': 8,
                 'tungro': 9}

inverse_map = {v:k for k,v in class_indices.items()}

In [19]:
m1_p_df = pd.DataFrame({"image_id":test_data_224.filenames,
                        "label":[inverse_map[k] for k in m1_predict_max]})
m2_p_df = pd.DataFrame({"image_id":test_data_224.filenames,
                        "label":[inverse_map[k] for k in m2_predict_max]})
m3_p_df = pd.DataFrame({"image_id":test_data_300.filenames,
                        "label":[inverse_map[k] for k in m3_predict_max]})

m1_p_df.image_id = m1_p_df.image_id.str.replace('./', '')
m2_p_df.image_id = m2_p_df.image_id.str.replace('./', '')
m3_p_df.image_id = m3_p_df.image_id.str.replace('./', '')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


In [21]:
m1_p_df.set_index('image_id', inplace=True)
m2_p_df.set_index('image_id', inplace=True)
m3_p_df.set_index('image_id', inplace=True)

In [23]:
m1_p_df['conf'] = m1_confidance
m2_p_df['conf'] = m2_confidance
m3_p_df['conf'] = m3_confidance

In [24]:
m1_p_df

,label,conf
image_id,,
200001.jpg,hispa,0.997981
200002.jpg,normal,0.999919
200003.jpg,blast,0.999951
200004.jpg,blast,0.999584
200005.jpg,blast,0.999992
...,...,...
203465.jpg,dead_heart,0.999972
203466.jpg,hispa,0.961185
203467.jpg,normal,0.999999


## ensemblling

In [9]:
s_full = pd.concat([s1,s2.drop('image_id', axis=1),s3.drop('image_id', axis=1)], axis=1).set_index('image_id')
s_full

,label,label,label
image_id,,,
200001.jpg,hispa,hispa,hispa
200002.jpg,normal,normal,normal
200003.jpg,blast,blast,blast
200004.jpg,blast,blast,blast
200005.jpg,blast,blast,blast
...,...,...,...
203465.jpg,dead_heart,dead_heart,dead_heart
203466.jpg,hispa,hispa,hispa
203467.jpg,normal,normal,normal


In [10]:
s_full_mode = ss.mode(s_full, axis=1)

In [11]:
s_df = pd.DataFrame()
s_df['label'] = s_full_mode[0].reshape(1,-1)[0]
s_df['count'] = s_full_mode[1]
s_df

,label,count
0,hispa,3
1,normal,3
2,blast,3
3,blast,3
4,blast,3
...,...,...
3464,dead_heart,3
3465,hispa,3
3466,normal,3
3467,bacterial_leaf_streak,3


In [12]:
s_full['most'] = s_full_mode[0].reshape(1,-1)[0]
s_full

,label,label,label,most
image_id,,,,
200001.jpg,hispa,hispa,hispa,hispa
200002.jpg,normal,normal,normal,normal
200003.jpg,blast,blast,blast,blast
200004.jpg,blast,blast,blast,blast
200005.jpg,blast,blast,blast,blast
...,...,...,...,...
203465.jpg,dead_heart,dead_heart,dead_heart,dead_heart
203466.jpg,hispa,hispa,hispa,hispa
203467.jpg,normal,normal,normal,normal


In [33]:
pd.concat([m1_p_df.loc[s_full.iloc[s_df[s_df['count']<2].index].index],
           m2_p_df.loc[s_full.iloc[s_df[s_df['count']<2].index].index],
           m3_p_df.loc[s_full.iloc[s_df[s_df['count']<2].index].index]], axis=1)

,label,conf,label,conf,label,conf
image_id,,,,,,
200107.jpg,blast,0.501823,brown_spot,0.544271,tungro,0.509404
200263.jpg,brown_spot,0.999755,dead_heart,0.375493,blast,0.962661
200519.jpg,brown_spot,0.983826,tungro,0.908766,blast,0.526320
200996.jpg,bacterial_panicle_blight,0.584124,normal,0.788273,dead_heart,0.965040
201312.jpg,tungro,0.879920,blast,0.335763,normal,0.997258
201845.jpg,bacterial_leaf_blight,0.533903,downy_mildew,0.803776,hispa,0.999665
201848.jpg,bacterial_leaf_blight,0.794102,hispa,0.379833,brown_spot,0.983907
201920.jpg,brown_spot,0.858595,tungro,0.584902,normal,0.999586
202159.jpg,tungro,0.986164,blast,0.629790,dead_heart,0.913759


In [15]:
s_full.iloc[s_df[s_df['count']<2].index].index

Index(['200107.jpg', '200263.jpg', '200519.jpg', '200996.jpg', '201312.jpg',
       '201845.jpg', '201848.jpg', '201920.jpg', '202159.jpg', '202267.jpg',
       '202379.jpg', '202423.jpg', '202736.jpg', '203100.jpg'],
      dtype='object', name='image_id')

In [36]:
s_final = s_full[['most']]
s_final.iloc[s_df[s_df['count']<2].index] = [['brown_spot'], ['brown_spot'], ['brown_spot'], ['dead_heart'], ['normal'], ['hispa'], ['brown_spot'],
                                             ['normal'], ['tungro'],['blast'],['bacterial_leaf_streak'],['bacterial_leaf_streak'],['tungro'],['tungro']]
s_final

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,most
image_id,
200001.jpg,hispa
200002.jpg,normal
200003.jpg,blast
200004.jpg,blast
200005.jpg,blast
...,...
203465.jpg,dead_heart
203466.jpg,hispa
203467.jpg,normal


In [37]:
s = s_final.rename(columns={'most':'label'}).reset_index()
s

,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast
...,...,...
3464,203465.jpg,dead_heart
3465,203466.jpg,hispa
3466,203467.jpg,normal
3467,203468.jpg,bacterial_leaf_streak


In [38]:
s.to_csv('submission_v11.csv', index=False)